In [1]:
import networkx as nx
from node2vec import Node2Vec
import pandas as pd

In [2]:
# create graph
df = pd.read_csv("../nijisanji_chat_graph.csv")

In [3]:
df.columns

Index(['source', 'target', 'edge_count', 'scaled_edge_count'], dtype='object')

In [4]:
df.columns = ['source', 'target', 'edge_count', 'weight']
G = nx.from_pandas_edgelist(df, "source", "target", ["weight"])

In [23]:
# node2vecを計算する前にpagerankなどを計算しておきたい
# エッジを消す前のフルのデータで
page_ranks = pd.DataFrame.from_dict(nx.pagerank(G), orient='index').reset_index()
page_ranks.columns = ["Label", "full_PageRank"]
page_ranks = page_ranks.sort_values("full_PageRank", ascending=False)
page_ranks.head(10)

,Label,full_PageRank
56,月ノ美兎,0.023508
80,舞元啓介,0.022857
22,リゼ・ヘルエスタ,0.020359
5,アンジュ・カトリーナ,0.020251
10,グウェル・オス・ガール,0.020069
63,椎名唯華,0.019416
81,花畑チャイカ,0.018830
30,剣持刀也,0.018371
73,社築,0.016769
25,三枝明那,0.016763


In [25]:
graph_feature = pd.read_csv("../graph_feature.csv")
pd.merge(graph_feature, page_ranks).to_csv("../graph_feature_add_full_page_rank.csv", index=None)

In [17]:
# 固有ベクトル中心性
eigenV_centrality = pd.DataFrame.from_dict(nx.eigenvector_centrality(G, weight="weight"), orient='index').reset_index()
eigenV_centrality.columns = ["streamer_name", "eigenVector_centrality"]
eigenV_centrality = eigenV_centrality.sort_values("eigenVector_centrality", ascending=False)
eigenV_centrality.head(10)

,streamer_name,eigenVector_centrality
56,月ノ美兎,0.226816
80,舞元啓介,0.226184
22,リゼ・ヘルエスタ,0.203307
5,アンジュ・カトリーナ,0.196968
10,グウェル・オス・ガール,0.191891
63,椎名唯華,0.190569
81,花畑チャイカ,0.184856
30,剣持刀也,0.179474
73,社築,0.165132
25,三枝明那,0.159658


In [41]:
# Precompute probabilities and generate walks - **ON WINDOWS ONLY WORKS WITH workers=1**
node2vec = Node2Vec(G, dimensions=128, walk_length=30, num_walks=200, workers=4)  # Use temp_folder for big graphs

Computing transition probabilities: 100%|██████████| 104/104 [00:07<00:00, 13.35it/s]


In [49]:
# Embed nodes
model = node2vec.fit(window=10, min_count=1, batch_words=4, seed=42)

In [50]:
model.wv.most_similar('月ノ美兎')

[('舞元啓介', 0.9683517217636108),
 ('文野環', 0.9535559415817261),
 ('轟京子', 0.9533289670944214),
 ('赤羽葉子', 0.9525494575500488),
 ('花畑チャイカ', 0.9523841142654419),
 ('ましろ', 0.9476977586746216),
 ('御伽原江良', 0.9439211487770081),
 ('緑仙', 0.943633496761322),
 ('夕陽リリ', 0.9429670572280884),
 ('成瀬鳴', 0.931716799736023)]

In [51]:
model.wv.most_similar('社築')

[('魔界ノりりむ', 0.9622323513031006),
 ('成瀬鳴', 0.9611464738845825),
 ('花畑チャイカ', 0.9546036720275879),
 ('鈴原るる', 0.9466467499732971),
 ('笹木咲', 0.9453818798065186),
 ('本間ひまわり', 0.9380985498428345),
 ('ニュイ・ソシエール', 0.9288491606712341),
 ('エクス・アルビオ', 0.9278891086578369),
 ('文野環', 0.9273958206176758),
 ('夢追翔', 0.9249404668807983)]

In [54]:
model.wv.most_similar('叶')

[('葛葉', 0.9757106304168701),
 ('不破湊', 0.962429940700531),
 ('椎名唯華', 0.9347176551818848),
 ('勇気ちひろ', 0.9159013032913208),
 ('ラトナ・プティ', 0.9092192649841309),
 ('エクス・アルビオ', 0.8829002380371094),
 ('イブラヒム', 0.8717691898345947),
 ('三枝明那', 0.8703913688659668),
 ('渋谷ハジメ', 0.861824631690979),
 ('グウェル・オス・ガール', 0.8347264528274536)]

In [55]:
model.wv.most_similar('リゼ・ヘルエスタ')

[('ドーラ', 0.935104489326477),
 ('御伽原江良', 0.9325144290924072),
 ('童田明治', 0.9321447610855103),
 ('文野環', 0.9286779165267944),
 ('鈴鹿詩子', 0.921648383140564),
 ('町田ちま', 0.9202717542648315),
 ('物述有栖', 0.9120903015136719),
 ('夕陽リリ', 0.9059736132621765),
 ('静凛', 0.9032505750656128),
 ('樋口楓', 0.8946034908294678)]

In [56]:
model.wv.most_similar('アンジュ・カトリーナ')

[('郡道美玲', 0.9656461477279663),
 ('エリー・コニファー', 0.9570568799972534),
 ('鈴鹿詩子', 0.9498031139373779),
 ('葉加瀬冬雪', 0.9413402080535889),
 ('空星きらめ', 0.9411152601242065),
 ('小野町春香', 0.9387507438659668),
 ('ドーラ', 0.9325311183929443),
 ('竜胆尊', 0.930375337600708),
 ('レヴィ・エリファ', 0.9266413450241089),
 ('山神カルタ', 0.9253100752830505)]

In [57]:
model.wv.most_similar('白雪巴')

[('相羽ういは', 0.9790965914726257),
 ('朝日南アカネ', 0.9771113395690918),
 ('小野町春香', 0.9461452960968018),
 ('北小路ヒスイ', 0.9450204372406006),
 ('黒井しば', 0.9445188045501709),
 ('山神カルタ', 0.9384509325027466),
 ('健屋花那', 0.9377341270446777),
 ('東堂コハク', 0.9370049238204956),
 ('周央サンゴ', 0.9360809326171875),
 ('空星きらめ', 0.9354572296142578)]

In [59]:
model.wv.most_similar('シェリン・バーガンディ')

[('長尾景', 0.9593351483345032),
 ('ベルモンド・バンデラス', 0.958264172077179),
 ('ルイス・キャミー', 0.9543248414993286),
 ('来栖夏芽', 0.954021692276001),
 ('アルス・アルマル', 0.9532814025878906),
 ('東堂コハク', 0.9498096108436584),
 ('西園チグサ', 0.9461591243743896),
 ('魔使マオ', 0.9458910226821899),
 ('メリッサ・キンレンカ', 0.945826530456543),
 ('桜凛月', 0.9420666694641113)]

In [60]:
model.wv.most_similar('ルイス・キャミー')

[('星川サラ', 0.9762992262840271),
 ('北小路ヒスイ', 0.9757951498031616),
 ('東堂コハク', 0.973049521446228),
 ('雪城眞尋', 0.9719136953353882),
 ('メリッサ・キンレンカ', 0.9718607068061829),
 ('来栖夏芽', 0.9711172580718994),
 ('フレン・E・ルスタリオ', 0.9657831192016602),
 ('ベルモンド・バンデラス', 0.9594865441322327),
 ('アルス・アルマル', 0.9591988325119019),
 ('シェリン・バーガンディ', 0.9543248414993286)]

In [61]:
model.wv.most_similar('イブラヒム')

[('三枝明那', 0.9712592363357544),
 ('弦月藤士郎', 0.9665244221687317),
 ('渋谷ハジメ', 0.9465838670730591),
 ('ラトナ・プティ', 0.9465606212615967),
 ('グウェル・オス・ガール', 0.941568911075592),
 ('黛灰', 0.9328826665878296),
 ('椎名唯華', 0.9311296939849854),
 ('勇気ちひろ', 0.9261093139648438),
 ('甲斐田晴', 0.9247403144836426),
 ('不破湊', 0.9171762466430664)]